In [42]:
import tensorflow as tf
import pandas as pd
import pickle
from sklearn.utils import shuffle
from sklearn.metrics import mean_absolute_error
import csv

In [43]:

"""
Global values
"""
folder = 'year2010_movie'

len_dtcgum = pickle.load(open('../data/'+folder+'/len_dtcgum.pkl', 'rb'))

FEATURE_LEN = 5

MOVIE_NUM = len_dtcgum[5]
DIRECTOR_NUM = len_dtcgum[0]
GENRE_NUM = len_dtcgum[3]
TOPIC_NUM = 50
CAST_NUM = len_dtcgum[2]
USER_NUM = len_dtcgum[4]

print('movie:', MOVIE_NUM)
print('user:', USER_NUM)
print('director:', DIRECTOR_NUM)
print('genre:', GENRE_NUM)
print('topic:', TOPIC_NUM)
print('cast:', CAST_NUM)

"""
MOVIE_NUM = 1682
DIRECTOR_NUM = 1139
GENRE_NUM = 24
TOPIC_NUM = 50
CAST_NUM = 2894
USER_NUM = 943
"""

movie: 3906
user: 3839
director: 3715
genre: 26
topic: 50
cast: 8065


'\nMOVIE_NUM = 1682\nDIRECTOR_NUM = 1139\nGENRE_NUM = 24\nTOPIC_NUM = 50\nCAST_NUM = 2894\nUSER_NUM = 943\n'

In [44]:
USER_NUM

3839

In [45]:
movie_rating = pickle.load(open('../data/'+folder+'/movie_rating.pkl', 'rb'))
movie_rating_test = pickle.load(open('../data/'+folder+'/movie_rating_test.pkl', 'rb'))
movie_rating_validation = pickle.load(open('../data/'+folder+'/movie_rating_validation.pkl', 'rb'))


In [46]:
def load_data(user_name, movie_name, rating_name, last_same):
    user = pickle.load(open('../data/'+folder+'/'+user_name+last_same+'.pkl', 'rb'))
    movie = pickle.load(open('../data/'+folder+'/'+movie_name+last_same+'.pkl', 'rb'))
    rating = pickle.load(open('../data/'+folder+'/'+rating_name+last_same+'.pkl', 'rb'))
    return shuffle(user, movie, rating)

#load training data
d_user, d_movie, d_rating = load_data("train_user", "train_movie", "train_rating", '_d')
t_user, t_movie, t_rating = load_data("train_user", "train_movie", "train_rating", '_t')
g_user, g_movie, g_rating = load_data("train_user", "train_movie", "train_rating", '_g')   
c_user, c_movie, c_rating = load_data("train_user", "train_movie", "train_rating", '_c')

#load validation data
d_user_v, d_movie_v, d_rating_v = load_data("train_user", "train_movie", "train_rating", '_d_v')
t_user_v, t_movie_v, t_rating_v = load_data("train_user", "train_movie", "train_rating", '_t_v')
g_user_v, g_movie_v, g_rating_v = load_data("train_user", "train_movie", "train_rating", '_g_v')        
c_user_v, c_movie_v, c_rating_v = load_data("train_user", "train_movie", "train_rating", '_c_v')


In [47]:
U = tf.Variable(initial_value=tf.truncated_normal([USER_NUM, FEATURE_LEN]), name='users')
D = tf.Variable(initial_value=tf.truncated_normal([FEATURE_LEN, DIRECTOR_NUM]), name='directors')
T = tf.Variable(initial_value=tf.truncated_normal([FEATURE_LEN, TOPIC_NUM]), name='topics')
G = tf.Variable(initial_value=tf.truncated_normal([FEATURE_LEN, GENRE_NUM]), name='genures')
C = tf.Variable(initial_value=tf.truncated_normal([FEATURE_LEN, CAST_NUM]), name='casts')

In [48]:
d_result = tf.matmul(U, D)
t_result = tf.matmul(U, T)
g_result = tf.matmul(U, G)
c_result = tf.matmul(U, C)

In [49]:
clip_U = U.assign(tf.maximum(tf.zeros_like(U), U))
clip_D = D.assign(tf.maximum(tf.zeros_like(D), D))
clip_T = T.assign(tf.maximum(tf.zeros_like(T), T))
clip_C = C.assign(tf.maximum(tf.zeros_like(C), C))
clip_G = G.assign(tf.maximum(tf.zeros_like(G), G))
clip = tf.group(clip_U, clip_D,clip_T,clip_C,clip_G )

In [50]:
g_result

<tf.Tensor 'MatMul_6:0' shape=(3839, 26) dtype=float32>

In [51]:
d_result_flatten = tf.reshape(d_result, [-1])
t_result_flatten = tf.reshape(t_result, [-1])
g_result_flatten = tf.reshape(g_result, [-1])
c_result_flatten = tf.reshape(c_result, [-1])

D_R = tf.gather(d_result_flatten, d_user * tf.shape(d_result)[1] + d_movie, name='director_user_rate')
T_R = tf.gather(t_result_flatten, t_user * tf.shape(t_result)[1] + t_movie, name='topic_user_rate')
G_R = tf.gather(g_result_flatten, g_user * tf.shape(g_result)[1] + g_movie, name='genre_user_rate')
C_R = tf.gather(c_result_flatten, c_user * tf.shape(c_result)[1] + c_movie, name='cast_user_rate')

In [52]:
d_diff_op = tf.subtract(D_R, d_rating, name='d_trainig_diff')
t_diff_op = tf.subtract(T_R, t_rating, name='t_trainig_diff')
g_diff_op = tf.subtract(G_R, g_rating, name='g_trainig_diff')
c_diff_op = tf.subtract(C_R, c_rating, name='c_trainig_diff')

d_diff_op_squared = tf.abs(d_diff_op, name="d_squared_difference")
t_diff_op_squared = tf.abs(t_diff_op, name="t_squared_difference")
g_diff_op_squared = tf.abs(g_diff_op, name="g_squared_difference")
c_diff_op_squared = tf.abs(c_diff_op, name="c_squared_difference")

#"""
d_base_cost = tf.reduce_sum(d_diff_op_squared, name="d_sum_squared_error")/ tf.cast(tf.shape(d_diff_op_squared)[0], tf.float32) * 10000 
t_base_cost = tf.reduce_sum(t_diff_op_squared, name="t_sum_squared_error")/ tf.cast(tf.shape(t_diff_op_squared)[0], tf.float32) * 10000
g_base_cost = tf.reduce_sum(g_diff_op_squared, name="g_sum_squared_error")/ tf.cast(tf.shape(g_diff_op_squared)[0], tf.float32) * 10000
c_base_cost = tf.reduce_sum(c_diff_op_squared, name="c_sum_squared_error")/ tf.cast(tf.shape(c_diff_op_squared)[0], tf.float32) * 10000

"""

d_base_cost = tf.reduce_sum(d_diff_op_squared, name="d_sum_squared_error")/30.
t_base_cost = tf.reduce_sum(t_diff_op_squared, name="t_sum_squared_error")/30.
g_base_cost = tf.reduce_sum(g_diff_op_squared, name="g_sum_squared_error")/30.
c_base_cost = tf.reduce_sum(c_diff_op_squared, name="c_sum_squared_error")/30.
"""

base_cost = d_base_cost + t_base_cost + g_base_cost + c_base_cost


In [53]:
lda = tf.constant(.001, name='lambda')

"""
u_norm = tf.reduce_sum(tf.abs(U, name='user_abs'), name='user_norm') / tf.cast(tf.shape(U)[0], tf.float32)
d_norm = tf.reduce_sum(tf.abs(D, name='director_abs'), name='director_norm')/ tf.cast(tf.shape(D)[1], tf.float32)
t_norm = tf.reduce_sum(tf.abs(T, name='topic_abs'), name='topic_norm')/ tf.cast(tf.shape(T)[1], tf.float32)
g_norm = tf.reduce_sum(tf.abs(G, name='genre_abs'), name='genre_norm')/ tf.cast(tf.shape(G)[1], tf.float32)
c_norm = tf.reduce_sum(tf.abs(C, name='cast_abs'), name='cast_norm')/ tf.cast(tf.shape(C)[1], tf.float32)
"""
u_norm = tf.reduce_sum(tf.abs(U, name='user_abs'), name='user_norm') 
d_norm = tf.reduce_sum(tf.abs(D, name='director_abs'), name='director_norm')
t_norm = tf.reduce_sum(tf.abs(T, name='topic_abs'), name='topic_norm')
g_norm = tf.reduce_sum(tf.abs(G, name='genre_abs'), name='genre_norm')
c_norm = tf.reduce_sum(tf.abs(C, name='cast_abs'), name='cast_norm')

norm_sums = u_norm + d_norm + t_norm + g_norm + c_norm
regularizer = tf.multiply(norm_sums, lda, 'regularizer')

In [54]:
#cost = tf.add(base_cost, regularizer)
cost = base_cost + regularizer

In [55]:
lr = tf.constant(.001, name='learning_rate')
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(lr, global_step, 10000, 0.96, staircase=True)
#learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_step = optimizer.minimize(cost, global_step=global_step)

In [56]:
"""
Generate Training RMSE
"""
g_user_test = g_user
g_item_test = g_movie
g_rate_test = g_rating
g_result_flatten = g_result_flatten
#genre rmse
g_test = tf.gather(g_result_flatten, g_user_test * tf.shape(g_result)[1] + g_item_test, name='g_test')
g_rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(g_rate_test, g_test))))

d_user_test = d_user
d_item_test = d_movie
d_rate_test = d_rating
d_result_flatten = d_result_flatten
#genre rmse
d_test = tf.gather(d_result_flatten, d_user_test * tf.shape(d_result)[1] + d_item_test, name='d_test')
d_rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(d_rate_test, d_test))))

t_user_test = t_user
t_item_test = t_movie
t_rate_test = t_rating
t_result_flatten = t_result_flatten
#genre rmse
t_test = tf.gather(t_result_flatten, t_user_test * tf.shape(t_result)[1] + t_item_test, name='t_test')
t_rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(t_rate_test, t_test))))

c_user_test = c_user
c_item_test = c_movie
c_rate_test = c_rating
c_result_flatten = c_result_flatten
#genre rmse
c_test = tf.gather(c_result_flatten, c_user_test * tf.shape(c_result)[1] + c_item_test, name='c_test')
c_rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(c_rate_test, c_test))))


rmse = tf.divide((g_rmse + d_rmse + t_rmse + c_rmse), 4)



In [57]:
"""
Generate Validation RMSE
"""
g_user_v = g_user_v
g_item_v = g_movie_v
g_rate_v = g_rating_v
g_result_flatten = g_result_flatten
#genre rmse
g_v = tf.gather(g_result_flatten, g_user_v * tf.shape(g_result)[1] + g_item_v, name='g_v')
g_rmse_v = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(g_rate_v, g_v))))

d_user_v = d_user_v
d_item_v = d_movie_v
d_rate_v = d_rating_v
d_result_flatten = d_result_flatten
#genre rmse
d_v = tf.gather(d_result_flatten, d_user_v * tf.shape(d_result)[1] + d_item_v, name='d_v')
d_rmse_v = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(d_rate_v, d_v))))

t_user_v = t_user_v
t_item_v = t_movie_v
t_rate_v = t_rating_v
t_result_flatten = t_result_flatten
#genre rmse
t_v = tf.gather(t_result_flatten, t_user_v * tf.shape(t_result)[1] + t_item_v, name='t_v')
t_rmse_v = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(t_rate_v, t_v))))

c_user_v = c_user_v
c_item_v = c_movie_v
c_rate_v = c_rating_v
c_result_flatten = c_result_flatten
#genre rmse
c_v = tf.gather(c_result_flatten, c_user_v * tf.shape(c_result)[1] + c_item_v, name='c_v')
c_rmse_v = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(c_rate_v, c_v))))


rmse_v = tf.divide((g_rmse_v + d_rmse_v + t_rmse_v + c_rmse_v), 4)

In [58]:
for i in range(len(g_user_v)):
    if g_user_v[i] < 0:
        print(i, g_item_v[i], g_user_v[i])
        break

In [59]:
#training_trend_f = open('../data/'+folder+'/trends.csv', 'w')
#csv_trend = csv.writer(training_trend_f)
#headers = ['round','train_total', 'train_g', 'train_d', 'train_t', 'train_c', 'valid_total', 'valid_g', 'valid_d', 'valid_t', 'valid_c']
#csv_trend.writerow(headers)
lowest_num = 0
lowest_rmse = 9999999
not_dec_num = 0

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
for i in range(100001):
    _, c, r = sess.run([training_step, cost, regularizer])
    if i%20 == 0:
        _rmse_v, _g_rmse_v, _d_rmse_v, _t_rmse_v, _c_rmse_v  = sess.run([rmse_v, g_rmse_v, d_rmse_v, t_rmse_v, c_rmse_v])
        sess.run(clip)
        if lowest_rmse > _rmse_v + 0.001:
            lowest_rmse = _rmse_v
            lowest_num = i
            not_dec_num = 0
        else:
            not_dec_num += 1
            if not_dec_num > 10:
                break
            
        print(i, c, "current rmsr: ", _rmse_v,  "lowest rmse: ", lowest_rmse, "lowest num: ", lowest_num)
        print(_g_rmse_v, _d_rmse_v, _t_rmse_v, _c_rmse_v)
        _rmse, _g_rmse, _d_rmse, _t_rmse, _c_rmse = sess.run([rmse, g_rmse, d_rmse, t_rmse, c_rmse])
        #w_row = [i, _rmse, _g_rmse, _d_rmse, _t_rmse, _c_rmse, _rmse_v, _g_rmse_v, _d_rmse_v, _t_rmse_v, _c_rmse_v]
        #csv_trend.writerow(w_row)
    if i%200 == 0:
        print()
        print("Traing RMSR(total, g, d, t, c):", i , _rmse, _g_rmse, _d_rmse, _t_rmse, _c_rmse)
        print()
#training_trend_f.close()

0 143075.0 current rmsr:  4.00055 lowest rmse:  4.00055 lowest num:  0
4.0215 3.96018 4.03845 3.98207

Traing RMSR(total, g, d, t, c): 0 3.11568 3.20509 3.0557 3.15191 3.05002

20 70071.6 current rmsr:  2.00604 lowest rmse:  2.00604 lowest num:  20
1.49574 2.40348 1.54388 2.58108
40 47586.3 current rmsr:  1.47398 lowest rmse:  1.47398 lowest num:  40
0.97578 1.80486 1.03529 2.07999
60 36506.6 current rmsr:  1.22522 lowest rmse:  1.22522 lowest num:  60
0.821555 1.49345 0.855154 1.73073
80 32060.4 current rmsr:  1.12809 lowest rmse:  1.12809 lowest num:  80
0.832282 1.33827 0.808323 1.53349
100 29972.8 current rmsr:  1.07598 lowest rmse:  1.07598 lowest num:  100
0.845423 1.24632 0.796075 1.41611
120 28734.4 current rmsr:  1.04154 lowest rmse:  1.04154 lowest num:  120
0.852659 1.18432 0.791829 1.33736
140 27890.8 current rmsr:  1.01696 lowest rmse:  1.01696 lowest num:  140
0.859138 1.1395 0.79003 1.27916
160 27284.4 current rmsr:  0.998129 lowest rmse:  0.998129 lowest num:  160
0.863

1380 22247.4 current rmsr:  0.844876 lowest rmse:  0.84558 lowest num:  1360
0.847494 0.853411 0.785868 0.89273
1400 22226.4 current rmsr:  0.844317 lowest rmse:  0.844317 lowest num:  1400
0.846727 0.852819 0.785928 0.891794

Traing RMSR(total, g, d, t, c): 1400 0.74562 0.68166 0.808144 0.6277 0.864976

1420 22191.9 current rmsr:  0.843806 lowest rmse:  0.844317 lowest num:  1400
0.846211 0.852259 0.785854 0.890902
1440 22157.8 current rmsr:  0.843351 lowest rmse:  0.844317 lowest num:  1400
0.84581 0.851709 0.785866 0.890021
1460 22135.1 current rmsr:  0.84283 lowest rmse:  0.84283 lowest num:  1460
0.845055 0.851162 0.785947 0.889157
1480 22102.6 current rmsr:  0.842078 lowest rmse:  0.84283 lowest num:  1460
0.843361 0.850654 0.785981 0.888316
1500 22061.1 current rmsr:  0.841496 lowest rmse:  0.841496 lowest num:  1500
0.842459 0.850155 0.785861 0.887509
1520 22054.1 current rmsr:  0.841078 lowest rmse:  0.841496 lowest num:  1500
0.842085 0.849635 0.785911 0.886682
1540 22015.5 c

2740 20630.7 current rmsr:  0.818787 lowest rmse:  0.819557 lowest num:  2680
0.803473 0.830595 0.785171 0.855909
2760 20590.6 current rmsr:  0.818409 lowest rmse:  0.818409 lowest num:  2760
0.802499 0.830396 0.785146 0.855596
2780 20574.3 current rmsr:  0.818384 lowest rmse:  0.818409 lowest num:  2760
0.802942 0.830182 0.785146 0.855263
2800 20557.8 current rmsr:  0.817961 lowest rmse:  0.818409 lowest num:  2760
0.801854 0.829956 0.785093 0.85494

Traing RMSR(total, g, d, t, c): 2800 0.704776 0.624799 0.759802 0.624071 0.810431

2820 20549.0 current rmsr:  0.817766 lowest rmse:  0.818409 lowest num:  2760
0.80164 0.829733 0.785097 0.854595
2840 20522.7 current rmsr:  0.81746 lowest rmse:  0.818409 lowest num:  2760
0.800882 0.829531 0.785137 0.854289
2860 20520.8 current rmsr:  0.817371 lowest rmse:  0.817371 lowest num:  2860
0.801133 0.8293 0.785101 0.853949
2880 20475.9 current rmsr:  0.817016 lowest rmse:  0.817371 lowest num:  2860
0.800174 0.829123 0.785113 0.853653
2900 2047

4100 19587.0 current rmsr:  0.806112 lowest rmse:  0.806112 lowest num:  4100
0.780833 0.819085 0.785655 0.838873
4120 19569.5 current rmsr:  0.805801 lowest rmse:  0.806112 lowest num:  4100
0.779882 0.818967 0.785661 0.838695
4140 19568.8 current rmsr:  0.805842 lowest rmse:  0.806112 lowest num:  4100
0.780306 0.818855 0.785693 0.838515
4160 19566.5 current rmsr:  0.805811 lowest rmse:  0.806112 lowest num:  4100
0.780473 0.818734 0.785707 0.838329
4180 19535.7 current rmsr:  0.805537 lowest rmse:  0.806112 lowest num:  4100
0.779723 0.818607 0.785665 0.838151
4200 19543.9 current rmsr:  0.805587 lowest rmse:  0.806112 lowest num:  4100
0.780199 0.818503 0.785669 0.837977

Traing RMSR(total, g, d, t, c): 4200 0.679981 0.588608 0.732316 0.619018 0.779982

4220 19525.1 current rmsr:  0.805331 lowest rmse:  0.806112 lowest num:  4100
0.779433 0.818398 0.785679 0.837813
4240 19520.8 current rmsr:  0.805281 lowest rmse:  0.806112 lowest num:  4100
0.77954 0.818288 0.785658 0.83764
4260 1

5460 18940.5 current rmsr:  0.798385 lowest rmse:  0.799352 lowest num:  5260
0.76661 0.812385 0.785682 0.828862


In [60]:
movie_feature_dict = pickle.load(open('../data/'+folder+'/movie_feature_dict.pkl', 'rb'))

In [61]:
def generate_matrix(row_num, col_num, user, movie, rating):
    # row is movie infor num
    # col is user num
    matr = [[0 for i in range(col_num)] for j in range(row_num)]
    for i in range(len(user)):
        matr[movie[i]][user[i]] = float(rating[i])
    
    return matr
        
    

In [62]:
user_v, director_v, topic_v, genre_v, cast_v = sess.run([U,D,T,G,C])

d_mat = generate_matrix(DIRECTOR_NUM, USER_NUM, d_user, d_movie, d_rating)
t_mat = generate_matrix(TOPIC_NUM, USER_NUM, t_user, t_movie, t_rating)
c_mat = generate_matrix(CAST_NUM, USER_NUM, c_user, c_movie, c_rating)
g_mat = generate_matrix(GENRE_NUM, USER_NUM, g_user, g_movie, g_rating)


In [63]:
len(d_mat)

3715

In [64]:
id_2_genre = pickle.load(open('../data/'+folder+'/id_2_genre.pkl', 'rb'))
id_2_director = pickle.load(open('../data/'+folder+'/id_2_director.pkl', 'rb'))
id_2_cast = pickle.load(open('../data/'+folder+'/id_2_cast.pkl', 'rb'))
genre_2_id = pickle.load(open('../data/'+folder+'/genre_2_id.pkl', 'rb'))
director_2_id = pickle.load(open('../data/'+folder+'/director_2_id.pkl', 'rb'))
cast_2_id = pickle.load(open('../data/'+folder+'/cast_2_id.pkl', 'rb'))

In [65]:

def get_rating_list(info_id_list, info_matrix, user_vector, user_id, matrix):
    result_list = []
    for _id in info_id_list:
        if matrix[_id][user_id] > 0:
            result_list.append(matrix[_id][user_id]) 
        else:
            
            _v = info_matrix[:, int(_id)]
            _rating = np.dot(user_vector, _v)
            result_list.append(_rating)
    return result_list

def get_rating_list_train(id_tup_list, matrix):
    result_list = []
    for tup in id_tup_list:
        user_id = tup[0]
        info_id = tup[1]
        result_list.append(matrix[info_id][user_id])

def gen_regr_data(m_id, user_rating_tups, is_training = True):
    x_train = []
    y_train = []
    user_id_train = []
    for tup in user_rating_tups:
        one_user_x = []
        rating = float(tup[1])
        y_train.append(rating)
        
        user_id = int(tup[0])
        user_id_train.append(user_id)
        feature_dict = movie_feature_dict[m_id]
        u_v = user_v[user_id]
        
        d_x = get_rating_list(feature_dict['d'], director_v, u_v,user_id, d_mat)
        g_x = get_rating_list(feature_dict['g'], genre_v, u_v, user_id, g_mat)
        t_x = get_rating_list(feature_dict['t'], topic_v, u_v, user_id, t_mat)
        c_x = get_rating_list(feature_dict['c'], cast_v, u_v, user_id, c_mat)
        #print(user_id, d_x, g_x, t_x, c_x)
        
        
        one_user_x = d_x + g_x + t_x + c_x
        #one_user_x = d_x + g_x + c_x
        x_train.append(one_user_x)
        
        feature_name = []
        feature_name += ["d_" + id_2_director[_id] for _id in feature_dict['d']]
        feature_name += ["g_" + id_2_genre[_id] for _id in feature_dict['g']]
        feature_name += ["t_" + str(_id) for _id in feature_dict['t']]
        feature_name += ["c_" + id_2_cast[_id] for _id in feature_dict['c']]
        feature_name += ["bias"]
        
        
    return x_train, y_train, feature_name, user_id_train
        
            
              

In [66]:
user_count_list = pickle.load(open('../data/'+folder+'/user_count_list.pkl', 'rb'))

In [67]:
name_list = ['d', 't', 'c', 'g']
divid_list = []
for name in name_list:
    summ = 0
    total = 0
    for user in user_count_list[name]:
        for key in user_count_list[name][user]:
            summ += user_count_list[name][user][key]
            total += 1

    divid_list.append(summ/total)

# Store result matrix

In [68]:
_d_result, _t_result, _g_result, _c_result =  sess.run([d_result, t_result, g_result, c_result])

In [69]:
pickle.dump(_d_result, open('../data/'+folder+'/ranking_matrix_director.pkl', 'wb'))
pickle.dump(_t_result, open('../data/'+folder+'/ranking_matrix_type.pkl', 'wb'))
pickle.dump(_g_result, open('../data/'+folder+'/ranking_matrix_genre.pkl', 'wb'))
pickle.dump(_c_result, open('../data/'+folder+'/ranking_matrix_cast.pkl', 'wb'))

In [70]:
from sklearn import linear_model
import numpy as np

def select_feature_index(feature, n):
    index_list = [0 for i in range(len(feature))]
    temp_f = [abs(feature[i]) for i in range(len(feature))]
    for i in (range(n)):
        min_index = temp_f.index(max(temp_f))
        index_list[min_index] = 1
        temp_f[min_index] = 0
    
    return index_list

def get_predict_rating_old(user_rating, features):
    return np.mean([user_rating[i] + features[i] for i in range(len(user_rating))])


def get_predict_rating(user_rating, features):
    weight_n = [float(abs(ur)) for ur in user_rating]
    summ = sum(weight_n)
    weight = [ur/summ for ur in weight_n]
    #weight = [1./len(weight_n) for ur in weight_n]
    
    weight_total = sum([weight[i]* (user_rating[i] + features[i]) for i in range(len(weight))])
    
    return weight_total

def get_predict_rating_select(user_rating, features, n):
    if len(user_rating) < n:
        return get_predict_rating_old(user_rating, features)
    else:
        index_list = select_feature_index(user_rating, n)
        account_list = []
        for i in range(len(user_rating)):
            if index_list[i] == 1:
                account_list.append(user_rating[i] + features[i])
        return np.mean(account_list)

def get_predict_rating_count(user_rating, features, user_id, feature_name):
    weight = []
    weight_sum = 0
    offset = 0.01
    for j in range(len(feature_name)-1):
        name = feature_name[j]
        dict_name = name.split('_')[0]
        feature_n = name.split('_')[1]
        try:
            if dict_name == 'd':
                feature_key = director_2_id[feature_n]
                offset = 1/divid_list[0]
            elif dict_name == 'c':
                feature_key = cast_2_id[feature_n]
                offset = 1/divid_list[2]
            elif dict_name == 'g':
                feature_key = genre_2_id[feature_n]
                offset = 1/divid_list[3]
            elif dict_name == 't':
                feature_key = int(feature_n)
                offset = 1/divid_list[1]
        except:
            feature_key = ''
        try:
            prop = user_count_list[dict_name][user_id][feature_key]
        except:
            prop = 1
        prop = offset * prop
        weight_sum += prop
        weight.append(prop)      
    
    weight = [w/float(weight_sum) for w in weight]
    result = 0
    for i in range(len(user_rating)):
        #name = feature_name[i]
        #dict_name = name.split('_')[0]
        #ppr = (user_rating[i] + features[i]) * weight[i]
        result += (user_rating[i] + features[i]) * weight[i]
    """
    print(user_id)
    for i in range(len(weight)):
        print(feature_name[i], end='')
        print(" %.2f %.2f %.2f" % (user_rating[i], features[i], weight[i]))
    
    """
    
    return result

# Equal weight

In [71]:
total_predict = np.array([])
total_rating = np.array([])
total_predict_v = np.array([])
total_rating_v = np.array([])
total_predict_tr = np.array([])
total_rating_tr = np.array([])
#train_rmse_compare_F = []
for m in range(0, MOVIE_NUM+1):
    if m not in movie_rating:
        print("not in:", m)
        continue
    user_rating_tups = movie_rating[m]
    x_train, y_train, feature_name, user_id_train = gen_regr_data(int(m), user_rating_tups)
    
    if len(y_train) < 5:
        print('less than 5:', m)
        continue
    #x_train = x_train[1:2]
    #y_train = y_train[1:2]
    
    #if len(y_train) > 50:
     #   continue
    
    if m in movie_rating_test:     
        user_rating_tups_test = movie_rating_test[m]
        x_test, y_test, _, user_id_test= gen_regr_data(int(m), user_rating_tups_test)
    else:
        x_test = []
        y_test = []
        user_id_test = []
    
    if m in movie_rating_validation:     
        user_rating_tups_validation = movie_rating_validation[m]
        x_valid, y_valid, _, user_id_valid= gen_regr_data(int(m), user_rating_tups_validation)
    else:
        x_valid = []
        y_valid = []
        user_id_valid = []
        
    print(len(x_train), len(x_test), len(x_valid))
    feature = [0 for i in range(len(feature_name)-1)]
    
    for j in range(len(feature_name)-1):
        name = feature_name[j]
        dict_name = name.split('_')[0]
        feature_n = name.split('_')[1]

        
        for i in range(len(x_train)):
            feature[j] += (y_train[i] - x_train[i][j])
        
        feature[j] /= len(x_train)
    
    #weight = [1 - abs(f) for f in feature] if len(feature) > 0 else []
    #select_indexs = select_feature_index(feature, 100)
    #select_indexs = [1 for i in range(len(feature))]
    #for i in range(len(feature)):
        #print(feature_name[i], 1/(1-feature[i]))
        #if select_indexs[i] == 1:
            #print(feature_name[i], feature[i], weight[i])
    #print()
    

    predict_t = []
    predict_v = []
    predict_tr = []
    
    for i in range(len(x_train)):
        cc = get_predict_rating_old(x_train[i], feature)
        #print("%.2f %.2f" %(cc, y_train[i]))
        #print()
        predict_tr.append(cc)
    
    for i in range(len(x_test)):
        cc = get_predict_rating_old(x_test[i], feature)
        #print("%.2f %.2f" %(cc, y_test[i]))
        #print()
        predict_t.append(cc)
    

    for i in range(len(x_valid)):
        cc = get_predict_rating_old(x_valid[i], feature)
        predict_v.append(cc)
        #print("%.2f %.2f" %(cc, y_valid[i]))
        #print()
     
    train_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_tr, y_train))))
    valid_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_v, y_valid))))
    test_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_t, y_test))))
    
    #print(m, train_rmse, valid_rmse, test_rmse)
    #train_rmse_compare_F.append([m, len(y_train), len(y_valid),len(y_test), train_rmse, valid_rmse, test_rmse])
    
    total_predict = np.concatenate((total_predict, predict_t), axis = 0)
    total_rating = np.concatenate((total_rating, y_test), axis = 0)
    total_predict_v = np.concatenate((total_predict_v, predict_v), axis = 0)
    total_rating_v = np.concatenate((total_rating_v, y_valid), axis = 0)
    total_predict_tr = np.concatenate((total_predict_tr, predict_tr), axis = 0)
    total_rating_tr = np.concatenate((total_rating_tr, y_train), axis = 0)
    
    #print(feature)
    #print(feature_name)
        
    

18 3 3
less than 5: 1
not in: 2
79 8 12
443 62 45
165 21 15
less than 5: 6
554 84 64
95 11 15
1295 139 154
54 11 7
11 1 1
321 49 47
570 73 69
132 20 17
42 3 6
17 3 5
552 63 61
76 8 10
177 21 24
120 16 18
619 68 76
148 16 19
684 72 86
88 10 14
958 111 107
291 42 31
227 26 26
110 15 13
347 55 41
371 55 40
79 8 10
43 5 12
240 25 37
191 27 26
18 2 1
1514 193 210
233 20 19
455 58 62
30 4 2
5 0 0
81 9 11
133 20 19
368 38 48
37 4 5
55 6 5
582 68 88
124 19 21
183 25 16
331 42 44
485 64 55
1349 152 167
59 8 5
19 7 4
15 2 2
69 11 11
222 21 34
20 4 2
18 2 1
121 

/Users/Suyixin/anaconda/envs/ws/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/Suyixin/anaconda/envs/ws/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


19 7
341 40 49
62 11 3
105 17 14
740 85 100
223 32 24
126 16 19
687 80 66
1084 134 157
62 6 8
395 54 47
633 78 80
less than 5: 71
45 8 7
43 3 7
5 0 2
328 36 40
23 5 3
521 56 59
1575 216 172
less than 5: 79
less than 5: 80
970 138 131
527 60 65
17 1 2
110 13 14
403 54 45
547 65 61
117 18 25
217 28 29
less than 5: 89
7 0 2
55 11 10
less than 5: 92
62 10 4
760 79 101
237 26 29
9 1 0
33 5 6
14 5 0
less than 5: 99
31 1 5
less than 5: 101
6 1 1
15 1 2
39 3 3
1454 185 187
73 10 7
1388 195 168
10 0 1
1245 171 145
974 119 149
161 14 19
not in: 112
298 34 33
31 3 1
80 8 10
26 4 4
11 2 1
9 0 0
119 10 15
less than 5: 120
394 49 51
98 7 7
215 34 29
11 2 2
13 2 1
10 0 1
23 1 2
1261 170 153
55 14 5
194 22 22
833 99 105
198 23 27
19 3 2
121 13 14
115 13 14
36 0 6
169 23 20
267 35 23
89 16 8
162 22 23
300 47 45
146 24 18
15 1 0
15 1 1
53 4 9
27 3 3
107 20 18
45 5 5
43 3 10
153 22 19
26 3 2
52 3 4
20 6 7
45 3 7
16 7 2
617 88 85
25 5 5
72 8 5
45 5 6
1057 126 129
85 7 11
5 0 0
12 1 2
27 7 5
11 2 2
43 3 10

23 0 1
less than 5: 919
less than 5: 920
13 2 2
19 0 0
5 0 1
34 4 0
60 9 6
19 2 2
393 48 44
1794 201 229
178 23 23
14 1 0
221 23 31
405 57 55
less than 5: 933
17 2 1
less than 5: 935
19 4 3
57 9 7
6 0 0
less than 5: 939
less than 5: 940
19 3 3
31 3 2
5 1 1
less than 5: 944
not in: 945
19 2 1
less than 5: 947
less than 5: 948
6 0 0
13 2 2
29 5 4
less than 5: 952
less than 5: 953
97 16 9
6 0 0
not in: 956
6 0 0
50 3 3
17 2 8
less than 5: 960
7 4 2
less than 5: 962
less than 5: 963
less than 5: 964
7 2 0
24 3 3
16 2 3
less than 5: 968
60 5 12
less than 5: 970
less than 5: 971
less than 5: 972
20 3 2
9 1 1
less than 5: 975
less than 5: 976
less than 5: 977
less than 5: 978
5 1 0
12 0 1
less than 5: 981
less than 5: 982
not in: 983
19 5 0
less than 5: 985
less than 5: 986
241 20 30
less than 5: 988
44 8 1
9 0 0
12 0 1
less than 5: 992
70 14 8
23 2 2
14 2 3
20 4 1
not in: 997
less than 5: 998
5 0 0
11 1 4
201 29 27
10 0 3
less than 5: 1003
less than 5: 1004
9 3 1
8 1 2
9 0 1
14 0 2
less than

12 4 3
less than 5: 1704
less than 5: 1705
less than 5: 1706
less than 5: 1707
76 12 7
less than 5: 1709
101 16 11
5 0 0
5 0 0
12 0 1
5 0 0
less than 5: 1715
194 25 20
11 0 4
not in: 1718
less than 5: 1719
less than 5: 1720
99 18 13
34 6 3
less than 5: 1723
10 1 0
5 0 0
86 16 7
32 9 2
less than 5: 1728
11 2 1
19 3 2
less than 5: 1731
64 12 10
not in: 1733
689 96 73
less than 5: 1735
less than 5: 1736
5 0 0
7 2 1
less than 5: 1739
33 7 10
less than 5: 1741
less than 5: 1742
not in: 1743
13 3 1
less than 5: 1745
8 2 1
6 1 0
153 11 19
5 0 0
7 1 1
474 72 58
26 2 3
less than 5: 1753
49 5 4
61 7 4
less than 5: 1756
less than 5: 1757
less than 5: 1758
104 12 8
less than 5: 1760
610 85 83
less than 5: 1762
less than 5: 1763
less than 5: 1764
629 85 91
15 2 1
less than 5: 1767
less than 5: 1768
9 2 1
5 1 1
less than 5: 1771
26 4 1
6 0 1
less than 5: 1774
19 1 2
10 2 0
less than 5: 1777
34 10 7
14 3 4
11 1 2
not in: 1781
11 2 4
less than 5: 1783
less than 5: 1784
11 1 4
8 1 2
less than 5: 1787
l

29 4 3
5 0 0
19 3 2
5 0 0
less than 5: 2400
6 1 1
6 0 0
less than 5: 2403
less than 5: 2404
less than 5: 2405
56 7 6
169 16 17
less than 5: 2408
5 0 0
less than 5: 2410
8 1 0
less than 5: 2412
7 0 1
341 40 42
31 4 4
9 0 0
11 0 0
less than 5: 2418
6 0 2
11 1 1
158 10 18
less than 5: 2422
973 127 126
less than 5: 2424
11 3 0
less than 5: 2426
18 3 2
less than 5: 2428
not in: 2429
38 4 3
less than 5: 2431
6 1 1
less than 5: 2433
less than 5: 2434
less than 5: 2435
less than 5: 2436
less than 5: 2437
less than 5: 2438
7 1 0
less than 5: 2440
less than 5: 2441
less than 5: 2442
less than 5: 2443
7 1 1
6 0 0
6 0 1
260 32 22
8 3 3
6 1 0
not in: 2450
not in: 2451
less than 5: 2452
8 1 1
less than 5: 2454
less than 5: 2455
30 5 4
less than 5: 2457
less than 5: 2458
6 2 0
32 3 2
8 0 1
8 0 1
18 3 1
5 0 0
15 2 2
not in: 2466
less than 5: 2467
less than 5: 2468
369 41 53
not in: 2470
5 3 0
198 28 19
not in: 2473
432 58 64
not in: 2475
501 70 69
5 0 0
less than 5: 2478
less than 5: 2479
less than 5:

10 2 2
less than 5: 3142
less than 5: 3143
less than 5: 3144
7 0 0
less than 5: 3146
less than 5: 3147
less than 5: 3148
6 1 1
less than 5: 3150
350 51 55
58 14 11
40 5 7
11 2 1
less than 5: 3155
16 1 1
less than 5: 3157
less than 5: 3158
less than 5: 3159
less than 5: 3160
less than 5: 3161
not in: 3162
not in: 3163
less than 5: 3164
less than 5: 3165
less than 5: 3166
less than 5: 3167
less than 5: 3168
less than 5: 3169
less than 5: 3170
less than 5: 3171
12 2 0
less than 5: 3173
less than 5: 3174
14 2 2
34 7 5
79 14 6
not in: 3178
less than 5: 3179
30 1 7
27 2 2
less than 5: 3182
less than 5: 3183
less than 5: 3184
9 2 1
less than 5: 3186
35 8 4
88 11 14
19 4 3
less than 5: 3190
17 3 6
270 30 28
89 16 17
less than 5: 3194
less than 5: 3195
10 1 2
52 2 4
not in: 3198
not in: 3199
less than 5: 3200
less than 5: 3201
less than 5: 3202
less than 5: 3203
less than 5: 3204
11 1 0
less than 5: 3206
less than 5: 3207
less than 5: 3208
less than 5: 3209
less than 5: 3210
less than 5: 3211
n

less than 5: 3703
less than 5: 3704
not in: 3705
less than 5: 3706
less than 5: 3707
5 0 0
not in: 3709
less than 5: 3710
not in: 3711
not in: 3712
not in: 3713
not in: 3714
not in: 3715
less than 5: 3716
less than 5: 3717
less than 5: 3718
less than 5: 3719
less than 5: 3720
less than 5: 3721
less than 5: 3722
8 1 1
less than 5: 3724
less than 5: 3725
less than 5: 3726
not in: 3727
less than 5: 3728
not in: 3729
7 0 0
not in: 3731
less than 5: 3732
less than 5: 3733
less than 5: 3734
less than 5: 3735
less than 5: 3736
not in: 3737
less than 5: 3738
less than 5: 3739
less than 5: 3740
less than 5: 3741
not in: 3742
less than 5: 3743
less than 5: 3744
not in: 3745
not in: 3746
not in: 3747
less than 5: 3748
not in: 3749
less than 5: 3750
less than 5: 3751
less than 5: 3752
17 4 1
less than 5: 3754
less than 5: 3755
5 0 1
less than 5: 3757
less than 5: 3758
less than 5: 3759
less than 5: 3760
11 2 1
not in: 3762
less than 5: 3763
not in: 3764
less than 5: 3765
less than 5: 3766
less tha

In [72]:
rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict, total_rating))))  
print("train rmse:", rmse)

train rmse: 0.753094147885


In [73]:
for i in range(len(total_predict)):
    if total_predict[i] < 0:
        total_predict[i] = 0.
    if total_predict[i] > 5:
        total_predict[i] = 5.

for i in range(len(total_predict_v)):
    if total_predict_v[i] < 0:
        total_predict_v[i] = 0.
    if total_predict_v[i] > 5:
        total_predict_v[i] = 5.
        
for i in range(len(total_predict_tr)):
    if total_predict_tr[i] < 0:
        total_predict_tr[i] = 0.
    if total_predict_tr[i] > 5:
        total_predict_tr[i] = 5.

        
rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict_tr, total_rating_tr))))  
print("train rmse:", rmse)

mae = mean_absolute_error(total_rating_tr, total_predict_tr)
print("train mae:", mae)

rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict_v, total_rating_v))))  
print("valid rmse:", rmse)

mae = mean_absolute_error(total_rating_v, total_predict_v)
print("valid mae:", mae)

rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict, total_rating))))  
print("test rmse:", rmse)

mae = mean_absolute_error(total_rating, total_predict)
print("test mae:", mae)
        


train rmse: 0.417237131541
train mae: 0.303732380244
valid rmse: 0.745083809266
valid mae: 0.556428198587
test rmse: 0.752955788395
test mae: 0.557459762873


# weight with number of featured movies

In [74]:
total_predict = np.array([])
total_rating = np.array([])
total_predict_v = np.array([])
total_rating_v = np.array([])
total_predict_tr = np.array([])
total_rating_tr = np.array([])
train_rmse_compare_E = []
movie_feature = {}
feature_name_dict = {}
dropped_movie = 0
for m in range(0, MOVIE_NUM+1):
    if m not in movie_rating:
        print('not in', m)
        dropped_movie += 1
        continue
    user_rating_tups = movie_rating[m]
    x_train, y_train, feature_name, user_id_train = gen_regr_data(int(m), user_rating_tups)
    
    if len(y_train) < 5:
        print('less than 5:', m)
        dropped_movie += 1
        continue
    #if len(y_train) > 50:
     #   continue
    
    if m in movie_rating_test:     
        user_rating_tups_test = movie_rating_test[m]
        x_test, y_test, _, user_id_test= gen_regr_data(int(m), user_rating_tups_test)
    else:
        x_test = []
        y_test = []
        user_id_test = []
    
    if m in movie_rating_validation:     
        user_rating_tups_validation = movie_rating_validation[m]
        x_valid, y_valid, _, user_id_valid= gen_regr_data(int(m), user_rating_tups_validation)
    else:
        x_valid = []
        y_valid = []
        user_id_valid = []
    
    if len(x_test) == 0:
        continue
        
    
    feature = [0 for i in range(len(feature_name)-1)]
    
    for j in range(len(feature_name)-1):
        prop_sum = 0
        name = feature_name[j]
        dict_name = name.split('_')[0]
        feature_n = name.split('_')[1]
        offset = 0.01
        try:
            if dict_name == 'd':
                feature_key = director_2_id[feature_n]
                offset = 1/divid_list[0]
            elif dict_name == 'c':
                feature_key = cast_2_id[feature_n]
                offset = 1/divid_list[2]
            elif dict_name == 'g':
                feature_key = genre_2_id[feature_n]
                offsedivid_listt = 1/divid_list[3]
            elif dict_name == 't':
                feature_key = int(feature_n)
                offset = 1/divid_list[1]
        except:
            feature_key = ''
        
        for i in range(len(x_train)):
            try:
                prop = user_count_list[dict_name][user_id_train[i]][feature_key]
            except:
                prop = 1
            prop = offset * prop
            #prop_sum += prop
            #feature[j] += (y_train[i] - x_train[i][j]) * prop
            prop_sum += 1
            feature[j] += (y_train[i] - x_train[i][j])
        
        feature[j] /= prop_sum
    feature_name_dict[m] = feature_name
    movie_feature[m] = feature
    #weight = [1 - abs(f) for f in feature] if len(feature) > 0 else []
    #select_indexs = select_feature_index(feature, 100)
    #select_indexs = [1 for i in range(len(feature))]
    #for i in range(len(feature)):
        #print(feature_name[i], 1/(1-feature[i]))
        #if select_indexs[i] == 1:
            #print(feature_name[i], feature[i], weight[i])
    #print()
    

    predict_t = []
    predict_v = []
    predict_tr = []
    
    for i in range(len(x_train)):
        cc = get_predict_rating_count(x_train[i], feature, user_id_train[i], feature_name)
        #print("%.2f %.2f" %(cc, y_train[i]))
        #print()
        predict_tr.append(cc)
    
    for i in range(len(x_test)):
        cc = get_predict_rating_count(x_test[i], feature, user_id_test[i], feature_name)
        #print("%.2f %.2f" %(cc, y_test[i]))
        #print()
        predict_t.append(cc)
    

    for i in range(len(x_valid)):
        cc = get_predict_rating_count(x_valid[i], feature, user_id_valid[i], feature_name )
        predict_v.append(cc)
        #print("%.2f %.2f" %(cc, y_valid[i]))
        #print()
        
        
    #train_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_tr, y_train))))
    #valid_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_v, y_valid))))
    #test_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_t, y_test))))
    
    #print(m, train_rmse, valid_rmse, test_rmse)
    
    #print(m, train_rmse, valid_rmse, test_rmse)
    #train_rmse_compare_E.append([m, len(y_train), len(y_valid),len(y_test), train_rmse, valid_rmse, test_rmse])
    
        
    total_predict = np.concatenate((total_predict, predict_t), axis = 0)
    total_rating = np.concatenate((total_rating, y_test), axis = 0)
    total_predict_v = np.concatenate((total_predict_v, predict_v), axis = 0)
    total_rating_v = np.concatenate((total_rating_v, y_valid), axis = 0)
    total_predict_tr = np.concatenate((total_predict_tr, predict_tr), axis = 0)
    total_rating_tr = np.concatenate((total_rating_tr, y_train), axis = 0)


less than 5: 1
not in 2
less than 5: 6
less than 5: 71
less than 5: 79
less than 5: 80
less than 5: 89
less than 5: 92
less than 5: 99
less than 5: 101
not in 112
less than 5: 120
less than 5: 167
less than 5: 168
less than 5: 188
less than 5: 191
less than 5: 193
less than 5: 197
less than 5: 201
less than 5: 206
not in 217
less than 5: 220
not in 228
less than 5: 231
less than 5: 234
less than 5: 257
less than 5: 268
less than 5: 270
less than 5: 275
less than 5: 291
not in 293
less than 5: 296
less than 5: 313
less than 5: 314
less than 5: 318
less than 5: 327
less than 5: 338
less than 5: 340
less than 5: 347
less than 5: 356
less than 5: 361
less than 5: 362
less than 5: 364
less than 5: 375
less than 5: 377
less than 5: 381
less than 5: 385
not in 386
less than 5: 389
less than 5: 390
less than 5: 402
less than 5: 403
less than 5: 409
less than 5: 428
less than 5: 433
less than 5: 435
less than 5: 439
less than 5: 440
less than 5: 443
less than 5: 451
less than 5: 452
less than 5

less than 5: 1658
less than 5: 1659
less than 5: 1660
less than 5: 1664
less than 5: 1666
less than 5: 1667
less than 5: 1668
less than 5: 1669
not in 1670
less than 5: 1672
less than 5: 1673
less than 5: 1674
not in 1675
less than 5: 1678
less than 5: 1679
less than 5: 1681
less than 5: 1683
less than 5: 1684
less than 5: 1686
less than 5: 1687
less than 5: 1688
less than 5: 1690
less than 5: 1692
less than 5: 1693
not in 1695
less than 5: 1696
less than 5: 1697
less than 5: 1698
less than 5: 1699
not in 1702
less than 5: 1704
less than 5: 1705
less than 5: 1706
less than 5: 1707
less than 5: 1709
less than 5: 1715
not in 1718
less than 5: 1719
less than 5: 1720
less than 5: 1723
less than 5: 1728
less than 5: 1731
not in 1733
less than 5: 1735
less than 5: 1736
less than 5: 1739
less than 5: 1741
less than 5: 1742
not in 1743
less than 5: 1745
less than 5: 1753
less than 5: 1756
less than 5: 1757
less than 5: 1758
less than 5: 1760
less than 5: 1762
less than 5: 1763
less than 5: 176

less than 5: 2564
not in 2566
less than 5: 2567
less than 5: 2570
not in 2571
less than 5: 2572
less than 5: 2575
less than 5: 2576
less than 5: 2577
less than 5: 2579
less than 5: 2581
less than 5: 2583
less than 5: 2584
less than 5: 2585
less than 5: 2586
not in 2587
less than 5: 2588
less than 5: 2589
less than 5: 2591
less than 5: 2593
less than 5: 2594
less than 5: 2596
less than 5: 2599
less than 5: 2600
less than 5: 2602
less than 5: 2604
less than 5: 2606
less than 5: 2608
less than 5: 2609
not in 2611
less than 5: 2613
less than 5: 2616
less than 5: 2619
less than 5: 2620
less than 5: 2621
less than 5: 2622
less than 5: 2625
less than 5: 2628
less than 5: 2629
less than 5: 2630
less than 5: 2632
less than 5: 2635
less than 5: 2636
less than 5: 2638
not in 2639
less than 5: 2640
not in 2641
less than 5: 2642
less than 5: 2643
less than 5: 2645
less than 5: 2646
less than 5: 2647
not in 2649
less than 5: 2652
less than 5: 2654
less than 5: 2655
less than 5: 2657
less than 5: 266

less than 5: 3278
not in 3279
not in 3280
less than 5: 3281
less than 5: 3283
less than 5: 3284
less than 5: 3286
less than 5: 3288
less than 5: 3289
less than 5: 3290
not in 3291
less than 5: 3292
not in 3293
not in 3294
less than 5: 3295
less than 5: 3296
less than 5: 3297
less than 5: 3298
less than 5: 3301
less than 5: 3302
less than 5: 3303
less than 5: 3304
less than 5: 3305
less than 5: 3306
less than 5: 3309
not in 3311
less than 5: 3313
not in 3314
less than 5: 3315
less than 5: 3316
less than 5: 3317
not in 3318
less than 5: 3319
less than 5: 3320
less than 5: 3321
not in 3322
less than 5: 3323
less than 5: 3324
less than 5: 3325
less than 5: 3326
not in 3327
less than 5: 3328
not in 3329
less than 5: 3330
not in 3331
less than 5: 3332
less than 5: 3334
less than 5: 3335
less than 5: 3336
less than 5: 3338
less than 5: 3339
less than 5: 3340
less than 5: 3341
less than 5: 3342
not in 3343
not in 3344
less than 5: 3345
less than 5: 3346
less than 5: 3347
less than 5: 3348
less

In [75]:
len(total_predict)

24679

In [76]:
index = 3
for i in range(len(movie_feature[index])):
    print(feature_name_dict[index][i], movie_feature[index][i])

d_Daniel Barnz 0.0031645569620253164
g_Drama -0.6841995482469624
g_Fantasy -0.4401856351042304
g_Romance -0.5916395489236694
t_1 -0.5619033390895868
t_46 -0.3031710412392418
c_Alex Pettyfer -0.06118143459915611
c_Justin Bradley -1.0680626566014164e-16
c_Mary-Kate Olsen -1.0680626566014164e-16


In [77]:
for i in range(len(total_predict)):
    if total_predict[i] < 0:
        total_predict[i] = 0.
    if total_predict[i] > 5:
        total_predict[i] = 5.

for i in range(len(total_predict_v)):
    if total_predict_v[i] < 0:
        total_predict_v[i] = 0.
    if total_predict_v[i] > 5:
        total_predict_v[i] = 5.
        
for i in range(len(total_predict_tr)):
    if total_predict_tr[i] < 0:
        total_predict_tr[i] = 0.
    if total_predict_tr[i] > 5:
        total_predict_tr[i] = 5.

        
rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict_tr, total_rating_tr))))  
print( rmse)

mae = mean_absolute_error(total_rating_tr, total_predict_tr)
print(mae)

rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict_v, total_rating_v))))  
print(rmse)

mae = mean_absolute_error(total_rating_v, total_predict_v)
print( mae)


rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict, total_rating))))  
print( rmse)

mae = mean_absolute_error(total_rating, total_predict)
print(mae)
        


0.540475014951
0.39428049773
0.742284780363
0.554742111054
0.751287636443
0.556780088977


# Get one user's information

In [84]:
user_id = 1
feature_dict = ['d','t','c', 'g']
one_user_count_list = {}
for type_name in feature_dict:
    one_user_count_list[type_name] = {}
for i in range(len(feature_dict)):
    dict_name = feature_dict[i]
    divid_value = divid_list[i]
    for feature_key in user_count_list[dict_name][user_id]:
        one_user_count_list[dict_name][feature_key] = user_count_list[dict_name][user_id][feature_key]/divid_value
    

In [85]:
user_movie_rating = {}
user_movie_info = {}

for m in range(0, MOVIE_NUM+1):
    if m not in movie_rating:
        print('not in', m)
        dropped_movie += 1
        continue
    user_rating_tups = movie_rating[m]
    x_train, y_train, feature_name, user_id_train = gen_regr_data(int(m), user_rating_tups)
    if len(y_train) < 5:
        print('less than 5:', m)
        dropped_movie += 1
        continue
    
    one_uesr_ratings = []
    one_user_info = []
    weight = []
    x_pred, _, feature_name, user_id_pred = gen_regr_data(int(m), [(user_id, 0)])
    if user_id in user_id_train:
        ind = user_id_train.index(user_id)
        real_rating = y_train[ind]
    else:
        real_rating = -1
    for j in range(len(feature_name)-1):
        prop_sum = 0
        name = feature_name[j]
        dict_name = name.split('_')[0]
        feature_n = name.split('_')[1]
        offset = 0.01
        try:
            if dict_name == 'd':
                feature_key = director_2_id[feature_n]
            elif dict_name == 'c':
                feature_key = cast_2_id[feature_n]
            elif dict_name == 'g':
                feature_key = genre_2_id[feature_n]
            elif dict_name == 't':
                feature_key = int(feature_n)
        except:
            feature_key = ''
        try:
            weight.append(one_user_count_list[dict_name][feature_key])
        except:
            # if user have never seen this information
            weight.append(0.1)

    weight_normal = [w/sum(weight) for w in weight]
    print('begin')
    try:
        one_pred_value = sum([weight_normal[i]*(x_pred[0][i] + movie_feature[m][i]) for i in range(len(x_pred[0]))])
    except:
        continue
    feature_name_dict
    """
    print("movie:", m)
    print("pred:", one_pred_value)
    print("real:", real_rating)
    
    for i in range(len(x_pred[0])):
        print(feature_name_dict[m][i], x_pred[0][i], movie_feature[m][i], weight_normal[i])
    """
    user_movie_rating[m] = (one_pred_value, real_rating)
    user_movie_info[m] = {}
    user_movie_info[m]['info_name'] = feature_name_dict[m][0:-1]
    user_movie_info[m]['info_rating'] = x_pred[0]
    user_movie_info[m]['movie_offset'] = movie_feature[m]
    user_movie_info[m]['weight'] = weight_normal
    



        

begin
less than 5: 1
not in 2
begin
begin
begin
less than 5: 6
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
less than 5: 71
begin
begin
begin
begin
begin
begin
begin
less than 5: 79
less than 5: 80
begin
begin
begin
begin
begin
begin
begin
begin
less than 5: 89
begin
begin
less than 5: 92
begin
begin
begin
begin
begin
begin
less than 5: 99
begin
less than 5: 101
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
not in 112
begin
begin
begin
begin
begin
begin
begin
less than 5: 120
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
begi

less than 5: 1110
begin
begin
begin
begin
begin
less than 5: 1116
less than 5: 1117
begin
begin
less than 5: 1120
less than 5: 1121
begin
less than 5: 1123
not in 1124
begin
begin
less than 5: 1127
less than 5: 1128
begin
less than 5: 1130
begin
begin
less than 5: 1133
begin
less than 5: 1135
less than 5: 1136
begin
begin
begin
less than 5: 1140
begin
begin
begin
begin
begin
begin
begin
begin
begin
begin
less than 5: 1151
not in 1152
begin
begin
begin
begin
less than 5: 1157
begin
less than 5: 1159
less than 5: 1160
less than 5: 1161
less than 5: 1162
less than 5: 1163
begin
begin
begin
begin
begin
begin
less than 5: 1170
less than 5: 1171
begin
less than 5: 1173
less than 5: 1174
less than 5: 1175
not in 1176
less than 5: 1177
less than 5: 1178
begin
begin
begin
less than 5: 1182
less than 5: 1183
less than 5: 1184
begin
begin
begin
less than 5: 1188
less than 5: 1189
not in 1190
not in 1191
begin
not in 1193
less than 5: 1194
begin
begin
begin
begin
less than 5: 1199
begin
not in 120

less than 5: 1960
less than 5: 1961
begin
begin
begin
begin
less than 5: 1966
not in 1967
begin
less than 5: 1969
less than 5: 1970
not in 1971
begin
less than 5: 1973
begin
begin
less than 5: 1976
less than 5: 1977
begin
less than 5: 1979
less than 5: 1980
begin
begin
begin
less than 5: 1984
begin
not in 1986
less than 5: 1987
less than 5: 1988
less than 5: 1989
less than 5: 1990
begin
begin
less than 5: 1993
begin
less than 5: 1995
begin
less than 5: 1997
begin
begin
less than 5: 2000
less than 5: 2001
less than 5: 2002
begin
begin
less than 5: 2005
not in 2006
less than 5: 2007
less than 5: 2008
not in 2009
less than 5: 2010
begin
less than 5: 2012
begin
less than 5: 2014
begin
less than 5: 2016
begin
begin
begin
less than 5: 2020
less than 5: 2021
less than 5: 2022
begin
less than 5: 2024
begin
less than 5: 2026
less than 5: 2027
less than 5: 2028
begin
less than 5: 2030
begin
less than 5: 2032
not in 2033
less than 5: 2034
less than 5: 2035
less than 5: 2036
less than 5: 2037
not 

less than 5: 2829
not in 2830
not in 2831
not in 2832
less than 5: 2833
less than 5: 2834
begin
not in 2836
begin
not in 2838
less than 5: 2839
less than 5: 2840
less than 5: 2841
begin
begin
less than 5: 2844
begin
begin
begin
less than 5: 2848
less than 5: 2849
less than 5: 2850
begin
not in 2852
less than 5: 2853
begin
begin
less than 5: 2856
less than 5: 2857
begin
begin
less than 5: 2860
less than 5: 2861
not in 2862
begin
begin
less than 5: 2865
begin
less than 5: 2867
less than 5: 2868
not in 2869
less than 5: 2870
less than 5: 2871
less than 5: 2872
less than 5: 2873
not in 2874
begin
not in 2876
begin
less than 5: 2878
begin
less than 5: 2880
less than 5: 2881
begin
begin
begin
less than 5: 2885
less than 5: 2886
less than 5: 2887
less than 5: 2888
not in 2889
not in 2890
less than 5: 2891
less than 5: 2892
less than 5: 2893
less than 5: 2894
not in 2895
less than 5: 2896
less than 5: 2897
less than 5: 2898
begin
not in 2900
not in 2901
less than 5: 2902
less than 5: 2903
less

In [86]:
for m in user_movie_rating:
    if user_movie_rating[m][1] > 0:
        print(m, user_movie_rating[m][1])
        length = len(user_movie_info[m]['info_rating'])
        for i in range((length)):
            print(user_movie_info[m]['info_name'][i], user_movie_info[m]['info_rating'][i] + user_movie_info[m]['movie_offset'][i], user_movie_info[m]['weight'][i])


        

353 5.0
d_Olivier Nakache 5.0 0.05886390217097165
d_Eric Toledano 5.0 0.05886390217097165
g_Biography 4.179979111710272 0.07964400236198407
g_Comedy 4.348082112247762 0.10520140610500882
g_Drama 4.798885463054781 0.26092326146948513
t_16 5.181397690298888 0.04841803305348131
t_30 4.688093987272223 0.09456098370953232
c_FranÂois Cluzet 5.000256147540983 0.12230187873273543
c_Omar Sy 5.001423041894354 0.09784150298618835
c_Anne Le Ny 5.000585480093677 0.07338112723964124
1547 3.5
d_Lee Daniels 3.521103896103896 0.08205909621683147
g_Biography 3.838491175797963 0.05551377509711334
g_Drama 4.399167281966244 0.18186975572860267
t_0 3.9842532497906404 0.052111536086812876
t_3 4.404840152572749 0.03270878155671303
t_6 4.200943531668004 0.059153827699789614
t_14 3.9669756897732533 0.05603163680489355
t_30 4.346351161476995 0.06591126797914985
t_32 4.060385944127734 0.1827200241123726
t_38 4.014519689010304 0.00824874844478287
t_44 4.158915248162308 0.005156862963278353
t_49 4.42510077910206 0

t_42 4.327006608510819 0.025406126184218385
t_44 4.117941739637654 0.0047131904837748855
c_Shailene Woodley 4.51540258329556 0.07791305165705832
c_Ansel Elgort 4.5 0.062330441325646656
c_Nat Wolff 4.496703296703297 0.04674783099423498
2469 4.0
d_Phil Lord 3.8787262872628725 0.040815406506493394
d_Christopher Miller 3.8787262872628725 0.040815406506493394
g_Action 3.7636315050179805 0.033793813902326594
g_Comedy 3.8043651805424057 0.07294518366721717
g_Crime 3.8754250535017283 0.05151496021696127
t_1 4.004649033806252 0.03818356724596031
t_2 3.6423636770590977 0.03723996214816052
t_15 4.119537291579742 0.017426028222676807
t_28 3.8971307667966104 0.001875138738706902
t_32 3.8789853987475063 0.18176637094116727
t_34 4.124539327108188 0.1703144427679413
t_38 3.740372562624342 0.008205696539820314
t_46 3.6971979003929034 0.0337363034809867
c_Jonah Hill 3.8614512448779026 0.15264434200223703
c_Channing Tatum 3.963853088854446 0.06784192977877203
c_Peter Stormare 4.0 0.05088144733407901
2488

In [87]:
best_movie = -1
best_rating = 0
for m in user_movie_rating:
    if user_movie_rating[m][1] < 0 and user_movie_rating[m][0] > best_rating:
        best_rating = user_movie_rating[m][0]
        best_movie = m

best_movie

699

In [88]:
best_movie = 1817
print('rating:',user_movie_rating[best_movie])
length = len(user_movie_info[best_movie]['info_rating'])
for i in range((length)):
    part = (user_movie_info[best_movie]['info_rating'][i] + user_movie_info[best_movie]['movie_offset'][i]) * user_movie_info[best_movie]['weight'][i]
    print(user_movie_info[best_movie]['info_name'][i], user_movie_info[best_movie]['info_rating'][i], user_movie_info[best_movie]['movie_offset'][i], user_movie_info[best_movie]['weight'][i], part)
    #print(user_movie_info[best_movie]['info_name'][i])
    
    #print(part)


rating: (4.264957009529021, -1)
d_Peter Jackson 3.94838 -0.011415525114155266 0.11858234975255284 0.466854327913
g_Adventure 5.0 0.1547664836228588 0.21920136605081927 1.1299318548831088
g_Fantasy 4.0 0.22598463443016575 0.18788688518641655 0.794007089808741
t_33 3.89116 0.09761546737909066 0.11858234975255284 0.472997831724
c_Ian McKellen 3.94758 -0.011415525114155266 0.11858234975255284 0.466759615924
c_Martin Freeman 3.94786 -0.009010110893672487 0.11858234975255284 0.467078556099
c_Richard Armitage 3.94691 -0.005951321998288801 0.11858234975255284 0.467327733177
